### 카테고리별 지원 파싱 (청소년청년지원)
청소년청년지원 (140~187페이지) 파싱

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

# 환경변수 로드
load_dotenv()

True

In [2]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [3]:
def extract_page_text(pdf_path, page_num):
    """PDF 페이지에서 텍스트 추출"""
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [4]:
def parse_welfare_with_langchain(text, page_num):
    """랭체인을 사용하여 복지 프로그램 정보 파싱"""
    
    # LLM 초기화 (재시도 설정 추가)
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    # 출력 파서 설정
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    # 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_messages([
        ("system", """청소년청년지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "청소년·청년 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "청소년 방과후아카데미", "청년도약계좌", "청년내일채움공제")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["구분", "지원내용", "금액"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["청소년", "교육지원", "월 30만원"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 청소년청년지원제도명만
- 대상: 연령, 소득기준, 학력, 취업상태 등 구체적 조건
- 내용: 지원금액, 지원기간, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """청소년청년지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "청소년·청년 지원" 제외
- 실제 청소년청년지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    # 체인 구성
    chain = prompt | llm | parser
    
    # 재시도 로직
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "청소년청년지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)  # 2초 대기 후 재시도
            else:
                return None

In [5]:
# 설정
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(140, 188))  # 청소년청년지원 섹션 (140~187페이지)

welfare_programs = []

In [6]:
# 페이지별 파싱 실행
for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    # 페이지 텍스트 추출
    page_text = extract_page_text(pdf_path, page_num)
    
    # 랭체인으로 파싱
    program = parse_welfare_with_langchain(page_text, page_num)
    
    if program:
        welfare_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f"{program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    # API 호출 간격 조절
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 140 파싱 중... (1/48)
아동통합서비스 지원(드림스타트사업) 추출 완료 (표 0개)
페이지 141 파싱 중... (2/48)
가정위탁아동 지원 추출 완료 (표 1개)
페이지 142 파싱 중... (3/48)
학대피해아동 보호 및 지원 추출 완료 (표 0개)
페이지 143 파싱 중... (4/48)
경계선지능아동 맞춤형 사례관리서비스 추출 완료 (표 0개)
페이지 144 파싱 중... (5/48)
자립준비청년(보호종료아동) 자립수당 추출 완료 (표 0개)
페이지 145 파싱 중... (6/48)
자립지원 전담기관 운영 추출 완료 (표 1개)
페이지 146 파싱 중... (7/48)
청소년치료재활센터 추출 완료 (표 0개)
페이지 147 파싱 중... (8/48)
청소년동반자 프로그램 추출 완료 (표 1개)
페이지 148 파싱 중... (9/48)
학교 밖 청소년 건강검진 추출 완료 (표 1개)
페이지 149 파싱 중... (10/48)
위기청소년 특별지원 추출 완료 (표 1개)
페이지 150 파싱 중... (11/48)
청소년성문화센터 추출 완료 (표 0개)
페이지 151 파싱 중... (12/48)
청소년 인터넷･스마트폰 과의존 치유 지원 추출 완료 (표 0개)
페이지 152 파싱 중... (13/48)
위(Wee)프로젝트 추출 완료 (표 1개)
페이지 153 파싱 중... (14/48)
지역사회 청소년 통합지원체계(청소년안전망) 추출 완료 (표 0개)
페이지 154 파싱 중... (15/48)
청소년쉼터 추출 완료 (표 0개)
페이지 155 파싱 중... (16/48)
청소년자립지원관 추출 완료 (표 1개)
페이지 156 파싱 중... (17/48)
청소년의 조화로운 성장과 역량 강화를 지원하는 프로그램 추출 완료 (표 0개)
페이지 157 파싱 중... (18/48)
청소년활동진흥센터 추출 완료 (표 0개)
페이지 158 파싱 중... (19/48)
청소년활동 지원 추출 완료 (표 0개)
페이지 159 파싱 중... (20/48)
이주배경청

In [7]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/청소년청년지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(welfare_programs, f, ensure_ascii=False, indent=2)

print(f"\n파싱 완료: {len(welfare_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


파싱 완료: 48개 프로그램
저장 위치: ../../data/outputs/청소년청년지원_2025.json


In [8]:
# 결과 미리보기 (타입 안정성 추가)
for program in welfare_programs:
    tables = program.get('tables', [])
    # tables가 dict로 들어오는 경우가 있어 리스트로 정규화
    if isinstance(tables, dict):
        tables = [tables]
    table_count = len(tables)

    title = program.get('title', '무제')
    page = program.get('page', '?')
    target = program.get('target') or ''
    content = program.get('content') or ''

    print(f"\n{title} (페이지 {page}, 표 {table_count}개)")
    print(f"대상: {target[:80]}{'...' if len(target) > 80 else ''}")
    print(f"내용: {content[:80]}{'...' if len(content) > 80 else ''}")

    if table_count > 0:
        first_table = tables[0]
        cols = first_table.get('columns') if isinstance(first_table, dict) else None
        if isinstance(cols, list):
            print(f"표: {cols[:3]}...")
        else:
            print("표: (컬럼 정보 없음)")


아동통합서비스 지원(드림스타트사업) (페이지 140, 표 0개)
대상: 12세 이하(초등학생 이하)의 취약계층 아동 및 가족(임산부 포함)
내용: 아동통합서비스 지원(드림스타트사업)은 12세 이하(초등학생 이하)의 취약계층 아동 및 가족(임산부 포함)을 대상으로, 가정방문을 통해 조사한 양...

가정위탁아동 지원 (페이지 141, 표 1개)
대상: 친부모가 직접 양육할 수 없어 자격을 갖춘 위탁가정에서 보호하고 있는 아동, 전문가정위탁, 일반가정위탁, 일시가정위탁
내용: 가정위탁아동을 위한 양육보조금, 디딤씨앗통장, 심리치료비, 교통비, 상해보험료, 전문아동보호비, 아동용품구입비 등 다양한 지원이 제공됩니다.
표: ['구분', '지원내용']...

학대피해아동 보호 및 지원 (페이지 142, 표 0개)
대상: 만 18세 미만 학대피해아동 및 그 가족
내용: 학대피해아동 및 가족에 대한 상담, 심리치료, 교육서비스 제공, 재학대 발생방지를 위한 모니터링 등 사후관리, 학대피해아동 보호 및 숙식 제공,...

경계선지능아동 맞춤형 사례관리서비스 (페이지 143, 표 0개)
대상: 아동복지시설 보호아동 중 경계선지능으로 의심되거나 종합심리검사 결과 경계선지능으로 진단받은 아동
내용: 경계선지능아동을 위해 선별검사 비용을 지원하고 맞춤형 사례관리 서비스를 제공합니다. 선별검사는 지능검사, HTP, 문장완성검사 등으로 구성되어 ...

자립준비청년(보호종료아동) 자립수당 (페이지 144, 표 0개)
대상: 아동복지시설, 가정위탁 보호종료 아동 중 보호종료일을 기준으로 과거 2년 이상 연속하여 보호를 받은 자로서 다음 사항 중 어느 하나에 해당하는 ...
내용: 자립수당으로 월 50만 원을 지급합니다. 자립준비청년은 주소지 읍면동 주민센터(행정복지센터) 또는 복지누리집(www.bokjiro.go.kr)에...

자립지원 전담기관 운영 (페이지 145, 표 1개)
대상: 아동복지시설, 가정위탁 보호종료 5년 이내 자립준비청년
내용: 아동복지시설이나 가정위탁에서 보호종료